### DuckDB vs Polars ###

- mesmo cálculo

- em DuckDB e Polars

- usando Parquet apenas (formato colunar ideal)

In [ ]:
# paths para Parquet

p100k = ratings_100k.with_suffix(".parquet")
p33m = ratings_33m.with_suffix(".parquet")

In [ ]:
# Funções de benchmark

def run_duckdb(path):
    t0 = time.time()
    duckdb.sql(f"""
        SELECT movieId, AVG(rating)
        FROM '{path}'
        GROUP BY movieId
    """).df()
    return round(time.time() - t0, 3)

def run_polars(path):
    t0 = time.time()
    (
        pl.scan_parquet(path)
          .groupby("movieId")
          .agg(pl.col("rating").mean())
          .collect()
    )
    return round(time.time() - t0, 3)


In [ ]:
# Comparação

df_duck_polars = pd.DataFrame([
    ["100k", run_duckdb(p100k), run_polars(p100k)],
    ["33M", run_duckdb(p33m), run_polars(p33m)],
], columns=["Dataset", "DuckDB (s)", "Polars (s)"])

df_duck_polars
